
# Report
# Assignment 7b

## Introduction:
What is your data and where is it from?

Data is from Kaggle, that describes socio-economic and health factors that determine the overall development of the country.

What are your research questions and hypotheses?

Investigate if:
1. Total health spending per capita is significanly different between countries in Africa and Europe.
1. The average number of years a new born child would live (if the current mortality patterns are to remain the same) is significanly different between countries in Africa and Europe.
1. Death of children under 5 years of age per 1000 live births is significanly different between countries in Africa and Europe.
1. Death of children under 5 years of age per 1000 live births is significanly different between countries in North America and South America.
1. Total health spending per capita is significanly different between countries in North America and South America.
1. The GDP per capita is significanly different between countries in North America and South America.

## Related work
will be added later in the end of the notebook

## Methodology

Describe what you did and the context for your data.

Perfomed Student's T-test for the specified sets of data

Explain why you will be using sample or population equations, as appropriate for your data

Since we have data for the certain period of time for the certain set of countris and we want to learn if the difference is significant - we would use sample equations.

## Process and Results

Istalling useful library to help preprocess countries

In [ ]:
!pip install pycountry_convert

Importing packages

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import pycountry_convert as pc
import seaborn as sns
from scipy.stats import t
from scipy import stats
"""for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))"""

Available types of data are shown below

In [ ]:
descr = pd.read_csv('/kaggle/input/unsupervised-learning-on-country-data/data-dictionary.csv')
for i in range(len(descr)):
    print(f"{descr.iloc[i,0]} - {descr.iloc[i,1]}")

### Read data

In [ ]:
data = pd.read_csv('/kaggle/input/unsupervised-learning-on-country-data/Country-data.csv')
data.head()

### Visualizing
Visualize dependancies. One can see big correlation between certain columns.

In [ ]:
sns.set(style="ticks")
sns.pairplot(data)

### Normality check

Strictly speaking before doing Student's T-test data has to be checked for normality.

We would use two test here for that reason: Shapiro-Wilk test and Anderson test

Shapiro-Wilk test

In [ ]:
for col in data.columns[1:-1]:
    _, p_v = stats.shapiro(data[col])
    if p_v > 0.05:
        print(f'data in {col} is normally distributed')
    else:
        print(f'data in {col} is NOT normally distributed')

Anderson test

If the returned statistic is larger than critical values then for the corresponding significance level, the null hypothesis that the data come from the chosen distribution can be rejected.

In [ ]:
 for col in data.columns[1:-1]:
    a_stat,crit_vals, sig_lvls = stats.anderson(data[col], dist='norm')
    print(f'Data in {col}')
    for crit,sig_lvl in zip(crit_vals,sig_lvls):
        if a_stat > crit:
            print(f'is NOT normal with {sig_lvl}% certainty')
        else:
            print(f'is normal with {sig_lvl}% certainty')

Well, none of our data is normally distributed. We would do Student's T-test anyways.

Actually the normality check should be done for each dataset we are comparing. But we would suffice here with checks for the whole dataset.

In [ ]:
#Convert all countty names to list 
countries = data.country.tolist()

In [ ]:
#define continents
continents = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'EU': 'Europe'
}
#sorry Antarctica

In [ ]:
#changing the names of some countries to the representation that pycountry_convert uses
countries = ["Congo, The Democratic Republic of the" if x=="Congo, Dem. Rep." else x for x in countries]
countries = ["Republic of the Congo" if x=='Congo, Rep.' else x for x in countries]
countries = ["Côte d'Ivoire" if x=="Cote d'Ivoire" else x for x in countries]
countries = ["Laos" if x=="Lao" else x for x in countries]
countries = ["Macedonia" if x=="Macedonia, FYR" else x for x in countries]
countries = ["Federated States of Micronesia" if x=="Micronesia, Fed. Sts." else x for x in countries]
countries = ["Saint Vincent and the Grenadines" if x=="St. Vincent and the Grenadines" else x for x in countries]

In [ ]:
#helping function to search for strings
for c in countries:
    m = re.search(".*Macedon.*",c)
    if m:
        print(m.group())

In [ ]:
#We acquire two letter codes for each country
alpha2_codes = [pc.country_name_to_country_alpha2(country) for country in countries]

In [ ]:
#for each country we find its continent
cont_code = []
for code in alpha2_codes:
    if code != "TL": #idk what is the matter with TL, have to manually assign continent to it
        cont_code.append(pc.country_alpha2_to_continent_code(code))
    else:
        cont_code.append(pc.country_alpha2_to_continent_code("CN"))#TL is in Asia, just like China

In [ ]:
#create list of continents using continent codes from the step above
by_continent = [continents[cont] for cont in cont_code]

In [ ]:
#add continents to dataset
data['continent'] = by_continent

Here we define a function that performs T-test and outputs values of interest

In [ ]:
def signif(set1,set2):    
    #count
    c1 = len(set1)
    c2 = len(set2)
    #mean
    m1 = set1.mean()
    m2 = set2.mean()
    #sample var
    v1 = set1.var()
    v2 = set2.var()
    #pooled var
    pooled_var = (v1*(c1-1)+v2*(c2-1))/(c1+c2-2)
    #pooled std
    pooled_std = pooled_var**0.5
    #df
    df = c1 + c2 - 2
    #t score
    t_score = abs(m1-m2)/(pooled_std*(1/c1+1/c2)**0.5)
    #p value
    p_value = 2*t.cdf(-t_score,df)
    #print values
    print(f'set 1 \nsize = {c1}\nmean = {m1}\nsample var = {v1}\n')
    print(f'set 2 \nsize = {c2}\nmean = {m2}\nsample var = {v2}\n')
    print(f'pooled var = {pooled_var}\npooled std = {pooled_std}\ndf = {df}\nT-score = {t_score}\np-value = {p_value}')
    return p_value

In [ ]:
set1 = data[data['continent']=='Africa']
set2 = data[data['continent']=='Europe']

In [ ]:
col = 'health'
print(f'Analyzing {col} in Africa and Europe\n')
p_value1 = signif(set1[col],set2[col])
if p_value1 < 0.005:
    print(f'\n\n************\nsets are significantly different')
else:
    print(f'\n\n************\nsets are NOT significantly different')

To chech our results we would use built-in function for T-Test from scipy

In [ ]:
stats.ttest_ind(set1.health,set2.health)

As one can see the results match

In [ ]:
col = 'life_expec'
print(f'Analyzing {col} in Africa and Europe\n')
p_value2 = signif(set1[col],set2[col])
if p_value2 < 0.005:
    print(f'\n\n************\nsets are significantly different')
else:
    print(f'\n\n************\nsets are NOT significantly different')

In [ ]:
col = 'child_mort'
print(f'Analyzing {col} in Africa and Europe\n')
p_value3 = signif(set1[col],set2[col])
if p_value3 < 0.005:
    print(f'\n\n************\nsets are significantly different')
else:
    print(f'\n\n************\nsets are NOT significantly different')

In [ ]:
set3 = data[data['continent']=='North America']
set4 = data[data['continent']=='South America']

In [ ]:
col = 'child_mort'
print(f'Analyzing {col} in North America and South America\n')
p_value4 = signif(set3[col],set4[col])
if p_value4 < 0.005:
    print(f'\n\n************\nsets are significantly different')
else:
    print(f'\n\n************\nsets are NOT significantly different')

In [ ]:
col = 'health'
print(f'Analyzing {col} in North America and South America\n')
p_value5 = signif(set3[col],set4[col])
if p_value5 < 0.005:
    print(f'\n\n************\nsets are significantly different')
else:
    print(f'\n\n************\nsets are NOT significantly different')

In [ ]:
col = 'gdpp'
print(f'Analyzing {col} in North America and South America\n')
p_value6 = signif(set3[col],set4[col])
if p_value6 < 0.005:
    print(f'\n\n************\nsets are significantly different')
else:
    print(f'\n\n************\nsets are NOT significantly different')

## Discussion

Which of your t-tests was most significant?

The average number of years a new born child would live (if the current mortality patterns are to remain the same) is significanly different between countries in Africa and Europe.

What are the implications of these results? 

Socio economic situation in Africa and Europe are enormously different. Children are dying...

Are there perhaps other reasons for the results?

Low spending on health.

What other research might you examine with the data?

We can also look at the imports and exports. Draw correlations between them and mortality rates

## References